<a href="https://colab.research.google.com/github/soumyabodavula/nlp-projects/blob/main/information_retrieval.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Assignment 4: Information Retrieval

In this assignment you will implement a sparse retriever (TF-IDF) and a dense retriever (DPR) that find answers for a QA task. For the latter, we will finetune a pretrained (compact) [BERT](https://arxiv.org/abs/1810.04805) model from the HuggingFace library.

Real retrievers operate on very large corpora of text, e.g., the entire Wikipedia, but in this assignment to save on computational resources we will instead build retrievers that only operate on a small set of text passages (~100) provided for each input query. Such models are also known as *rerankers* and are often the second step of commercial retrieval systems.

**Warning**: Do not start this project the day before it is due!  Some parts require 20 minutes or more to run, so debugging and tuning can take a significant amount of time.

**Grading Rubric**
- 70% results
 - 20% preds_tfidf.txt (5% correctness + 15% meets target)
 - 20% preds_inbatch.txt (meets target)
 - 20% preds_hardneg.txt (meets target)
 - 10% preds_hardneg.txt (improvement over target)
  
- 30% writeup
 - 12.5% clarity
 - 12.5% correctness
 - 5% interestingness of ideas


TA contact for this assignment:
Sanxing Chen (sanxing.chen@duke.edu)

## Setup

In [ ]:
import json
import random
import re
import torch
import torch.nn as nn
import torch.optim as optim
import transformers
import unicodedata
import numpy as np
import scipy as sp
from scipy.sparse import csr_matrix
from sklearn.preprocessing import normalize
from transformers import AutoTokenizer, AutoModel
from tqdm import notebook as tqdm
import torch.nn.functional as F

np.random.seed(42)

We will use [BERT-Mini](https://huggingface.co/google/bert_uncased_L-4_H-256_A-4) for this assignment which is a relatively small pretrained model (~11M parameters), since it easily fits in the memory of Colab GPUs and training and inference are both usually fast.

In [ ]:
model_checkpoint = "google/bert_uncased_L-4_H-256_A-4"

In [ ]:
try:
    assert torch.cuda.is_available()
    device = torch.device("cuda")
except:
    device = torch.device("cpu")
print("Using device:", device)

Using device: cuda


The following code helps wrap the output text for better readability. You can ignore this.

In [ ]:
from IPython.display import HTML, display

def set_css():
  display(HTML('''
  <style>
    pre {
        white-space: pre-wrap;
    }
  </style>
  '''))
get_ipython().events.register('pre_run_cell', set_css)

## Data

We will use the [CuratedTREC](https://trec.nist.gov/data/qa.html) QA data which consists of questions paired with regex patterns specifying the expected answers to those questions. In addition to the questions, for each of them, we also have 100 passages from Wikipedia, which are all somewhat relevant to the question, but only some of them actually contain the answer.

First lets download the data.

In [ ]:
!gdown 1-FSEPWVOX2l7BCvjuIa3SwW37osJEBhH
!gdown 1-AAu2LBaSjK754zHydc4BBybYgb_GV2e
!gdown 1L3KNqeGQqmHob7v7oDPiyb-foqQSMHlA

Downloading...
From: https://drive.google.com/uc?id=1-FSEPWVOX2l7BCvjuIa3SwW37osJEBhH
To: /content/train.jsonl
100% 73.1M/73.1M [00:00<00:00, 128MB/s]
Downloading...
From: https://drive.google.com/uc?id=1-AAu2LBaSjK754zHydc4BBybYgb_GV2e
To: /content/dev.jsonl
100% 7.54M/7.54M [00:00<00:00, 58.1MB/s]
Downloading...
From: https://drive.google.com/uc?id=1L3KNqeGQqmHob7v7oDPiyb-foqQSMHlA
To: /content/test.jsonl
100% 44.9M/44.9M [00:00<00:00, 96.0MB/s]


The first thing we need to do is tokenize the questions and passages in all the above files. We will use the Huggingface Tokenizer class for this. Note that tokenizers are specific to the pretrained model that we use (and are hence loaded from the model checkpoint). While the pretrained model will only be used later for the dense retriever, we will use the same tokenizer throughout.

Note that we do not need to tokenize the answers as these will only be used for evaluation.

Running the cell below will take a few minutes as the passages need to be tokenized.

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint, use_fast=True)

def tokenize_qa(filename):
  with open(filename) as f:
    data = []
    for line in tqdm.tqdm(f):
      item = json.loads(line.strip())
      item["question_indices"] = tokenizer(item["question"])["input_ids"]
      item["passage_indices"] = tokenizer(item["passages"])["input_ids"]
      data.append(item)
  print("Read and tokenized %d items from %s" % (len(data), filename))
  return data

train_data = tokenize_qa("train.jsonl")
dev_data = tokenize_qa("dev.jsonl")
test_data = tokenize_qa("test.jsonl")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/383 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

0it [00:00, ?it/s]

Read and tokenized 1125 items from train.jsonl


0it [00:00, ?it/s]

Read and tokenized 116 items from dev.jsonl


0it [00:00, ?it/s]

Read and tokenized 694 items from test.jsonl


We can inspect what a typical example in the train and dev sets looks like. Note that the answers are regular expressions which need to be matched to a string / passage returned by the model.

When retrieving small passages of text, one common trick is to prepend the title of the page on which the passage appears to the passage itself. This sometimes helps resolve referential ambiguities in the passage. We have already done this for the passages provided: you will notice that each passage has the following structure "*page_title* || *passage_text*". These passages were capped at 100 words, so they might contain incomplete sentences.

In [ ]:
index = random.choice(range(len(dev_data)))
print("Question:", dev_data[index]["question"])
print("Answer regex:", dev_data[index]["answers"])
print("-----------------------------------------")
print("Random passage:", random.choice(dev_data[index]["passages"]))

Question: What movie did Madilyn Kahn star in with Gene Wilder?
Answer regex: ['Young\\s+Frankenstein']
-----------------------------------------
Random passage: What Did You Expect? (film) || live. The film is being distributed by MVD Entertainment, and was released on DVD in November 2012. It is also available on iTunes, Hulu, and Video-on-Demand. Writing in Punk News, John Gentile said, "The wilder songs, like 'Audiowhore' where bassist Matt Gentling just gets completely down, stomping around like a T-rex, are nearly berserk, with the band approaching a Stooges-type thrash. 'What Did You Expect?' could pass for Fugazi's wilder side. Gentling just goes nuts on the bass, which is wild. Likewise, the mid-tempo songs like "Freezing Point" have an inherent urgency." What Did You Expect? (film) What Did You


As always we will iterate over the data in batches. For retrieval reranking our batches will consist of queries and the associated passage lists. For the latter we will use a 3-dimensional tensor with the shape `(batch_size, num_passages, max_passage_length)`.

Also, the pretrained model we will use has a max length it can process (`tokenizer.model_max_length`), so we will truncate the passages to be shorter than this maximum length.

In [ ]:
def make_batch(batch_indices):
  """Convert a list of variable length texts into a batch.

  Args:
    texts: A list of list of token indices.

  Returns:
    A LongTensor of size (batch_size, max_sequence_length) containing the
    subword indices for the texts, where max_sequence_length is the length
    of the longest text and batch_size is the number of sentences in the batch.
    Empty slots at the end of shorter sequences should be filled with padding
    tokens. The tensor should be located on the device defined at the beginning
    of the notebook.
  """
  return torch.nn.utils.rnn.pad_sequence(
      [torch.LongTensor(item) for item in batch_indices],
      padding_value=tokenizer.pad_token_id,
      batch_first=True)

def make_batch_iterator(dataset, batch_size, shuffle=False):
  """Make a batch iterator that yields source-target pairs.

  Args:
    dataset: A list of dicts with the keys `question_indices` and `passages_indices`.
    batch_size: An integer batch size.
    shuffle: A boolean indicating whether to shuffle the examples.

  Yields:
    question_batch: batch_size x max_question_length
    passage_batch: batch_size x num_passages x max_passage_length
    example_batch: batch_size list of complete examples
  """

  if shuffle:
    random.shuffle(dataset)

  for start_index in range(0, len(dataset), batch_size):
    example_batch = dataset[start_index:start_index + batch_size]
    num_exs = len(example_batch)
    # tokenize questions.
    question_indices = [example["question_indices"] for example in example_batch]
    question_batch = make_batch(question_indices)
    # Each example has exactly 100 passages so we can flatten and reshape later.
    passage_indices = [psg_idx for example in example_batch for psg_idx in example["passage_indices"]]
    passage_batch = make_batch(passage_indices)
    passage_batch = passage_batch[:, :tokenizer.model_max_length]
    passage_batch = passage_batch.view(num_exs, len(dataset[0]["passages"]), -1)
    # print('printing.........')
    # print(question_batch[0])
    # print(passage_batch[0])
    # print(example_batch[0])

    yield question_batch, passage_batch, example_batch

## Evaluation

We will evaluate our rerankers by measuring recall@K. This checks whether at least one of the top-K passages after reranking contain the answer or not (as judged by the regex expression). The overall recall@K is computed as the fraction of questions for which one of the top-K passages contain the answer. We will focus on recall@5 and recall@20 throughout this assignment.

You don't need to modify the evaluation code below.

In [ ]:
class Reranker:
  """Interface for different rerankers to subclass."""
  def fit(self, data_iterator):
    raise NotImplementedError

  def rerank(self, query_batch, passage_batch):
    raise NotImplementedError

def _normalize(text):
  return unicodedata.normalize("NFD", text)

def regex_match(text, pattern):
  """Test if a regex pattern is contained within a text."""
  try:
    pattern = re.compile(pattern, flags=re.IGNORECASE + re.UNICODE + re.MULTILINE)
  except BaseException:
    return False
  return pattern.search(text) is not None

def has_answer(text, answers):
  """Checks if any of the answers are in the text."""
  for single_answer in answers:
    single_answer = _normalize(single_answer)
    if regex_match(text, single_answer):
      return True
  return False

def in_top_K(answers, ranked_passages, K):
  """Checks whether any of the top-K passages matches the answer regex.

  Args:
    answers: A list of regex patterns for the target answers.
    ranked_passages: A list of passage strings ranked in decreasing order of
      relevance.
    K: A list of top-K values to check. These must be in ascending order.

  Returns:
    A list, same size as K, with booleans indicating whether the answer was in
    the top-K or not. Note that once we have a True for one particular value of
    K, all subsequent values will also be True.
  """
  top_hit = None
  for i, psg in enumerate(ranked_passages):
    if has_answer(psg, answers):
      top_hit = i+1
      break
  if top_hit is None:
    return [False] * len(K)
  out = []
  for k in K:
    out.append(k >= top_hit)
  return out

@torch.no_grad()
def evaluate(data_iterator, model, K=[5, 20], verbose=False):
  """Computes recall@K for the rerankings produced by the model.

  Args:
    data_iterator: An iterator over the data as produced by the make_batch_iterator
      above.
    model: A Reranker model which implements the rerank method (see below).

  Returns:
    The recall@K scores of the reranker.
  """
  # if model is a torch module, put it in eval mode.
  if isinstance(model, nn.Module):
    model.eval()

  recalls = [0.] * len(K)
  total = 0
  for question_batch, passage_batch, example_batch in tqdm.tqdm(data_iterator):
    ranking_indices, ranking_scores = model.rerank(question_batch, passage_batch)
    for i in range(len(example_batch)):
      ranked_passages = [example_batch[i]["passages"][j] for j in ranking_indices[i]]
      topK = in_top_K(example_batch[i]["answers"], ranked_passages, K)
      recalls = [r+int(k) for r, k in zip(recalls, topK)]
      total += 1
  recalls = [r / total for r in recalls]
  if verbose:
    print("\t".join(["K"] + [str(k) for k in K]))
    print("-" * (12 * len(K)))
    print("\t".join(["Rec@K"] + ["%.3f" % r for r in recalls]))
  return recalls

@torch.no_grad()
def save_predictions(data_iterator, model, output_file):
  """Save the rank scores of passages for each question in the data.

  Args:
    data_iterator: An iterator over the data as produced by the make_batch_iterator
      above.
    model: A Reranker model which implements the rerank method (see below).
    output_file: A file to save the predictions to.
  """

  # if model is a torch module, put it in eval mode.
  if isinstance(model, nn.Module):
    model.eval()

  print('Saving predictions to', output_file)

  with open(output_file, "w") as f:
    for question_batch, passage_batch, example_batch in tqdm.tqdm(data_iterator):
      ranking_indices, ranking_scores = model.rerank(question_batch, passage_batch)
      # convert numpy to python lists
      ranking_indices = ranking_indices.tolist()
      ranking_scores = ranking_scores.tolist()
      for i in range(len(example_batch)):
        f.write(' '.join([str(x) for x in ranking_indices[i]]) + '\n')

## Random Reranker

Let's define a trivial reranker which simply returns a random ordering of the documents and check its recall.

In [ ]:
class TrivialReranker(Reranker):
  def rerank(self, query_batch, passage_batch):
    num_passages = passage_batch.shape[1]
    indices = np.zeros(passage_batch.shape[:2], dtype=np.int32)
    scores = np.zeros(passage_batch.shape[:2], dtype=np.int32)
    for i in range(passage_batch.shape[0]):
      indices[i, :] = np.random.permutation(passage_batch.shape[1])
    return indices, scores

dev_iterator = make_batch_iterator(dev_data, 16)
trivial_reranker = TrivialReranker()

In [ ]:
_ = evaluate(dev_iterator, trivial_reranker, verbose=True)

0it [00:00, ?it/s]

K	5	20
------------------------
Rec@K	0.422	0.741


The trivial reranker already gets ~35-40\% recall@5 and ~75\% recall@20. This is because there are several passages for each question which contain the correct answer and even a random ordering can sometimes place (at least) one of them in the top K.

These numbers will serve as a baseline to compare our other rerankers to.

## TFIDF

Next, lets implement a TF-IDF reranker. Recall that TF-IDF (Term Frequency-Inverse Document Frequency) is a numerical score that reflects how important a word is to a document. The TF component measures how frequently a term appears in a document, while the IDF component measures how unique or rare a term is across multiple documents.

Mathematically, the TF-IDF score for a term in a document can be calculated as:

\begin{gathered}
\operatorname{tf-idf}(t, d)=\operatorname{tf}_{t, d} \cdot \operatorname{idf}_t,
\end{gathered}


where TF is computed from the frequency of the term in the document:

\begin{gathered}
\mathrm{tf}_{t, d}=\log _{10}(\#(t, d)+1),
\end{gathered}

and IDF is calculated as:

\begin{gathered}
\operatorname{idf}_t=\log _{10} \frac{1 + N}{1 + \sum_{d^{\prime}} 1\left(t \in d^{\prime}\right)}.
\end{gathered}

Here, $N$ is the total number of documents in the corpus, and $\sum_{d^{\prime}} 1\left(t \in d^{\prime}\right)$ is the number of documents containing the term (document frequency). To avoid a division by zero if the term is not in the corpus, we adopt the common practice to increase both the numerator and the denominator by 1.

Given L2-normalized TF-IDF vectors for a query and a document, we compute the similarity score between them by taking the dot-product between them.

TF-IDF vectors are the same size as the vocabulary but they are very sparse, i.e., most entries are zero. Hence, we will use [Sparse Matrices](https://docs.scipy.org/doc/scipy/reference/sparse.html) from Scipy for storing the document and query vectors across a batch and taking the dot product between them. These data structures only store the non-zero elements of the sparse matrix and are hence very efficient both in terms of memory and computation. In particular, we will use the compressed sparse row (CSR) matrix representation which you can read more about [here](https://docs.scipy.org/doc/scipy/reference/generated/scipy.sparse.csr_matrix.html).

Please complete the `fit` and `vectorize` functions below. The `fit` method takes in an iterator over the training data and computes the IDF scores for the entire vocabulary. The `vectorize` method takes in a batch of token indices and computes the TF-IDF vector for each element in the batch. For both of these, you might want to (but don't have to) first implement a common utility for computing a sparse vector of term counts in the document, which can then be used to compute the IDF scores as well as the TF-IDF vectors.

A correctly implemented TF-IDF reranker should give you a recall@5 above 0.55 and a recall@20 above 0.81.

In [ ]:
class TFIDFReranker(Reranker):

  def __init__(self, vocab_size):
    self.vocab_size = vocab_size # 30522
    self.term_freqs = np.zeros((self.vocab_size,), dtype=np.int32) # count of docs that each vocab term appears in
    self.num_docs = 0


  def fit(self, data_iterator):
    """Compute inverse document frequences (IDF) from the provided data.

    Args:
      data_iterator: An iterator over the data as produced by the make_batch_iterator
        function above.
    """

    ### YOUR CODE HERE !!!!!
    # 1) count the number of documents each term in the vocabulary appears in
    # 2) use the counts to compute inverse document frequencies (DF)

    # precompute document frequency for each term in corpus
    for item in tqdm.tqdm_notebook(data_iterator): # (question_batch, passage_batch, example_batch)
      q_batch, p_batch, e_batch = item
      # print(q_batch.shape) (batch_size, max_question_length)
      # print(p_batch.shape) (batch_size, num_passages=100, max_passage_length)
      # print(len(e_batch))  (batch_size)

      # reshape p_batch to (batch_size * num_passages, max_passage_length)
      p_flat = p_batch.reshape(-1, p_batch.shape[-1])

      for passage in p_flat: # should have length of max_passage_length
        # add to number of docs that each unique term appears in
        unique_terms = np.unique(passage)
        self.term_freqs[unique_terms] += 1
        self.num_docs += 1

    # The code below should fill out the `idf_vals` vector with the non-negative
    # IDF value of each term in the vocabulary
    # idf_vals = np.zeros((self.vocab_size,), dtype=np.float32)
    idf_vals = np.log10((1 + self.num_docs) / (1 + self.term_freqs))

    ### END YOUR CODE HERE !!!!!

    # We will create a diagonal sparse matrix of size V x V to store the IDF values
    # so that they can be easily multiplied with sparse term frequency vectors to
    # get the TFIDF later.
    self.idf = csr_matrix(
        (idf_vals, (np.arange(self.vocab_size), np.arange(self.vocab_size))),
        shape=(self.vocab_size, self.vocab_size))

  def vectorize(self, indices):
    """Convert a batch of token indices to TFIDF vectors.

    Args:
      indices: (batch_size x max_len) Matrix of token indices, optionally ending
        with padding tokens.

    Returns:
      A sparse matrix of size (batch_size x vocab_size) containing the TFIDF vectors.
    """

    ### YOUR CODE HERE !!!!!

    # This code should compute the term frequencies of every vocab item in
    # a document. The result should be a sparse CSR matrix `term_frequencies`
    # of size batch_size x vocab_size, where each row contains the term frequencies
    # tf(t, d) of the corresponding batch element d.

    term_frequencies_list = []
    for document in indices: # batch_size = num_documents?
      term_counts = np.bincount(document[document != 0], minlength=self.vocab_size)
      term_counts += 1
      term_frequencies_list.append(np.log10(term_counts))

    term_frequencies = csr_matrix(term_frequencies_list)

    ### END YOUR CODE HERE !!!!!

    # multiply term_frequencies (batch_size x vocab_size) with the IDFs
    # (vocab_size x vocab_size) to get TF-IDFs.
    tfidfs = term_frequencies.dot(self.idf)

    # lastly we apply L2 normalization within each document
    return normalize(tfidfs, norm='l2', axis=1)

  def rerank(self, query_batch, passages_batch):
    """Compute TFIDF scores between queries and their associated passages.

    Args:
      query_batch: (batch_size x max len) A batch of query indices.
      passages_batch: (batch_size x num_psgs x max_len) A batch of passages for
        each query.

    Returns:
      indices: (batch_size x num_psgs) Indices which sort passages in descending
        order of TFIDF scores.
      scores: (batch_size x num_psgs) Sorted TFIDF scores.
    """
    indices = np.zeros(passages_batch.shape[:2], dtype=np.int32)
    scores = np.zeros(passages_batch.shape[:2], dtype=np.float32)
    for i in range(query_batch.shape[0]):
      query_tfidf = self.vectorize(query_batch[[i], :]) # 1 x vocab_size
      passages_tfidf = self.vectorize(passages_batch[i, :, :]) # num_psgs x vocab_size
      sc = (passages_tfidf.dot(query_tfidf.transpose())).toarray().squeeze()
      idx = np.argsort(sc)[::-1]
      indices[i, :] = idx
      scores[i, :] = sc[idx]
    return indices, scores

# Train the TF-IDF reranker.
tfidf_reranker = TFIDFReranker(tokenizer.vocab_size)
train_iterator = make_batch_iterator(train_data, 16)
tfidf_reranker.fit(train_iterator)

# Evaluate on the dev set.
dev_iterator = make_batch_iterator(dev_data, 16)
_ = evaluate(dev_iterator, tfidf_reranker, verbose=True)

save_predictions(make_batch_iterator(test_data, 4), tfidf_reranker, "preds_tfidf.txt")

0it [00:00, ?it/s]

0it [00:00, ?it/s]

K	5	20
------------------------
Rec@K	0.552	0.819
Saving predictions to preds_tfidf.txt


0it [00:00, ?it/s]

KeyboardInterrupt: 

## Dense Retriever

Next let's explore a dense retriever, similar to the one introduced by [Karpukhin et al., 2020](https://arxiv.org/abs/2004.04906), which can capture more semantics of the query and documents. The dense retriever adopts a bi-encoder architecture, essentially using a pretrained BERT model to separately encode the queries and the passages, and then compare their embeddings to rank by similarity.

In this assignment, we will use a shared encoder for both the query and passage, which means that a single model will be used for producing the embeddings of both. You can however try using different encoders in the improvement section. We will use **dot-product** as the similarity measurement.

### Off-the-shelf DPR

First, let's use the pretrained model directly without any finetuning. We will compute the embedding of both the query and the list of passages by taking the output from BERT corresponding to the `[CLS]` token. Then the score of each passage will be defined as the dot product between these embeddings.

Please complete the `rerank` function, the implementation should be very similar to the TF-IDF `rerank`.

The pretrained model should give you a recall@5 above 0.59 and recall@20 above 0.83.

In [ ]:
class PretrainedDualEncoderModel(nn.Module):
  def __init__(self):
    super().__init__()
    # Load the pretrained model checkpoint.
    self.encoder = AutoModel.from_pretrained(model_checkpoint).to(device)

  def vectorize(self, indices):
    """Returns the [CLS] token embeddings after passing through the encoder."""
    # The tokenizer already adds a [CLS] token to the beginning of the tokenized
    # indices, so we just need to take the hidden state output at position 0.
    mask = indices != tokenizer.pad_token_id
    outputs = self.encoder(input_ids=indices.to(device), attention_mask=mask.to(device))
    return outputs.last_hidden_state[:, 0, :]

  def rerank(self, query_batch, passages_batch):
      """Rerank passages based on pretrained model embeddings.

      Args:
        query_batch: (batch_size x max_len) A batch of query indices.
        passages_batch: (batch_size x num_psgs x max_len) A batch of passages for
          each query.

      Returns:
        indices: (batch_size x num_psgs) Indices which sort passages in descending
          order of relevance scores.
        scores: (batch_size x num_psgs) Sorted relevance scores.
      """
      indices = np.zeros(passages_batch.shape[:2], dtype=np.int32)
      scores = np.zeros(passages_batch.shape[:2], dtype=np.float32)

      ### YOUR CODE HERE !!!!!

      # get the scores for each query and its associated passages, and use them
      # to rerank.

      indices = np.zeros(passages_batch.shape[:2], dtype=np.int32)
      scores = np.zeros(passages_batch.shape[:2], dtype=np.float32)

      for i in range(query_batch.shape[0]):  # each query
        query_vector = self.vectorize(query_batch[[i], :])
        passages_vector = self.vectorize(passages_batch[i, :, :])  # num_psgs x vocab_size
        scores[i, :] = torch.matmul(query_vector, passages_vector.T).squeeze().cpu().numpy()
        idx = np.argsort(scores[i, :])[::-1]
        indices[i, :] = idx
        scores[i, :] = scores[i, idx]

      return indices, scores

      ### END YOUR CODE HERE !!!!!

# Evaluate on the dev set.
pretrained_reranker = PretrainedDualEncoderModel()
dev_iterator = make_batch_iterator(dev_data, 16)
_ = evaluate(dev_iterator, pretrained_reranker, verbose=True)


pytorch_model.bin:   0%|          | 0.00/45.1M [00:00<?, ?B/s]

0it [00:00, ?it/s]

K	5	20
------------------------
Rec@K	0.595	0.836


### In-batch Finetuned DPR

The pretrained reranker already shows an improvement in recall@K over the TFIDF approach. Next, let's try to finetune the pretrained BERT model on our training data to see if that helps.

Recall that the loss we use to optimize the dense retriever is:

\begin{gathered}
\mathcal{L} = \sum_{q, p^+, \{p_i^-\}} -\log \frac{\exp\left(\operatorname{sim}(q, p^+)\right)}{\exp\left(\operatorname{sim}(q, p^+)\right) + \sum_{p_i^-}\exp\left(\operatorname{sim}(q, p_i^+)\right)}.
\end{gathered}

Note that this loss can be interpreted as a softmax over a vector of scores between the query and the positive passage as well as all the negative passages. For this part, we will only use **in-batch negatives** to train the reranker, i.e., the $p_i^-$ will be the positive passages $p^+$ for the other queries in the same batch. As a result the number of negatives per query will be `batch_size - 1`.

In the class below, first implement the `forward` method which takes a batch of queries and a single positive passage per query and returns the log probabilities across all passages in the batch. Note that this probability needs to be normalized w.r.t the in-batch negatives, i.e., the positives of the all the other questions in the batch.

Then, implement the `fit` method which trains the network by iterating over the data, **randomly** selecting a single positive for each query in the batch and calling the forward method for computing the logits. To train the network you can use the `NLLLoss()` criterion by passing in the index of the positive passage for the query as the target.

In [ ]:
class InBatchDualEncoderModel(PretrainedDualEncoderModel):
  def __init__(self):
    super().__init__()

  def forward(self, query_batch, passage_batch):
    """Encode queries and passages and compute log probabilities.

    Args:
      query_batch: (batch_size x max_len) token indices for the queries.
      passage_batch: (batch_size x max_len) token indices for the passages.

    Returns:
      log_probs: (batch_size x batch_size) normalized log probabilities obtained by
        taking a softmax of the inner products between each query and all of
        the passages.
    """
    # Run the forward pass on queries and passages.
    # You will probably want to use the `log_softmax` function to compute the log
    # probabilities once you have the scores.

    ### YOUR CODE HERE !!!!!
    query_batch = query_batch.to(dtype=torch.long)
    passage_batch = passage_batch.to(dtype=torch.long)
    query_vector = self.vectorize(query_batch) # need to vectorize each query individually?
    passage_vector = self.vectorize(passage_batch)
    inner_products = torch.matmul(query_vector, passage_vector.T)  # (batch_size x batch_size)

    # normalize probability wrt in-batch negatives (other queries' positives)
    log_probs = F.log_softmax(inner_products, dim=1) # along row/query
    diagonal_scores = torch.diagonal(log_probs)
    log_probs = torch.diag_embed(diagonal_scores)
    ### END YOUR CODE HERE !!!!!

    return log_probs

  def fit(self, train_data, dev_data, model_file, n_epochs=5, batch_size = 16):
    dev_data_iterator = list(make_batch_iterator(dev_data, 4))
    criterion = nn.NLLLoss()
    optimizer = optim.Adam(self.parameters(), lr=5e-5)
    best_recall = 0.
    for epoch in range(n_epochs):
      recall_at_5 = evaluate(dev_data_iterator, self, K=[5], verbose=True)[0]
      if recall_at_5 > best_recall:
        print(
            "Obtained a new best recall@5 of {:.2f}, saving model "
            "checkpoint to {}...".format(recall_at_5, model_file))
        torch.save(self.state_dict(), model_file)
        best_recall = recall_at_5
      self.train()
      running_loss = 0.
      train_data_iterator = list(make_batch_iterator(train_data, batch_size, shuffle=True))
      pbar = tqdm.tqdm(enumerate(train_data_iterator))
      for i, (query_batch, passage_batch, example_batch) in pbar:
        optimizer.zero_grad()
        n = passage_batch.shape[0] # batch size

        # `query_batch`: (batch_size x max len) A batch of query indices.
        # `passages_batch`: (batch_size x num_psgs x max_len) A batch of passages for each query.
        # `example_batch`: (batch_size) A list of dict,
         # {..., 'positives': [] }  containing indices of positive passages.

        # Sample a random positive for each query to get batch_size passages,
        # a positive to one query is a negative to the other query
        # calculate nll loss based on where the positive is among the batch.
        # Think carefully about what the `target` argument should look like! We
        # have already initialized the NLLLoss() `criterion` for you above.
        ### YOUR CODE HERE !!!!!
        rand_passages_batch = torch.zeros(passage_batch.shape[0], passage_batch.shape[2])
        targets = torch.zeros(passage_batch.shape[0])
        for i in range(query_batch.shape[0]): # for each query/batch index
          pos_idx = example_batch[i]['positives'] # indices of positive passages for query i
          rand_pos_idx = pos_idx[torch.randint(len(pos_idx), (1,)).item()]
          rand_pos_passage = passage_batch[i, rand_pos_idx, :]
          targets[i] = i
          rand_passages_batch[i] = rand_pos_passage

        targets = targets.to(dtype=torch.long, device=query_batch.device)
        logits = self.forward(query_batch, rand_passages_batch) # (batch_size, batch_size)
        logits = logits.to(device=query_batch.device)
        loss = criterion(logits, targets)
        ### END YOUR CODE HERE !!!!!

        loss.backward()
        optimizer.step()
        lr = optimizer.param_groups[0]['lr']

        # print statistics
        running_loss += loss.item()
        pbar.set_description("Epoch: {}, Loss: {:.2f}, Best R@5: {:.3f}, lr: {:.2g}".format(epoch, running_loss / (i + 1), best_recall, lr))
    print("Reloading best model checkpoint from {}...".format(model_file))
    self.load_state_dict(torch.load(model_file))

Lets train the model above.
You should able to get a recall@5 of 0.70 and recall@20 of 0.85 with in-batch negatives, but you might need to run multiple times in case you get an unlucky seed the first time.

In [ ]:
# Train.
inbatch_reranker = InBatchDualEncoderModel()
inbatch_reranker.fit(train_data, dev_data, "inbatch_dualencoder.pt")

# Evaluate on the dev set.
_ = evaluate(make_batch_iterator(dev_data, 4), inbatch_reranker, verbose=True)

save_predictions(make_batch_iterator(test_data, 4), inbatch_reranker, "preds_inbatch.txt")

### Hard Negative DPR

Finetuning the pretrained model improves recall, but the benefit of training with only in-batch negatives quickly diminishes. The task of distinguishing between positives for one query and positives for a completely different query is much easier than our actual task of finding positives among negatives for the same query. The model might just learn a task of differentiating between very distant topics.

To improve the dense reranker, we can reduce this training objective mismatch by introducing "hard negatives", i.e., passages that are related to the query but do not actually contain the answer. Essentially for each query, **in addition to the in-batch negatives**, we further sample one negative passage for each query from the provided passages (**excluding those labeled as positive**). Then the softmax in the loss will be over all positives as well as all hard negatives for all queries in the batch. This results in total 2 x batch\_size terms in the denominator above, where exactly one passage is a positive.

Please also complete the `forward` and `fit` functions below. The `forward` function will now take in two sets of passages (`pos_batch` and `neg_batch`) and compute log probabilities across both of these. The `fit` function will train the model by computing the standard DPR loss using these 2 x batch\_size passages.

In [ ]:
class HardNegativeDualEncoderModel(PretrainedDualEncoderModel):
  def __init__(self):
    super().__init__()

  def forward(self, query_batch, pos_batch, neg_batch):
    """Encode queries and passages and compute log probabilities.

    Args:
      query_batch: (batch_size x max_len) token indices for the queries.
      pos_batch: (batch_size x max_len) token indices for a single positive
        passage per query.
      neg_batch: (batch_size x max_len) token indices for a single negative
        passage per query.

    Returns:
      log_probs: (batch_size x 2 * batch_size) normalized log probabilities obtained by
        taking a softmax of the inner products between the queries and each of
        the positive and negative passages across the batch.
    """
    # Run the forward pass on queries and passages.

    ### YOUR CODE HERE !!!!!

    # You will probably want to use the `log_softmax` function to compute the log
    # probabilities once you have the scores.

    query_batch = query_batch.to(dtype=torch.long)
    pos_batch = pos_batch.to(dtype=torch.long)
    neg_batch = neg_batch.to(dtype=torch.long)
    query_vector = self.vectorize(query_batch) # (batch_size x max_len)
    pos_vector = self.vectorize(pos_batch) # (batch_size x max_len)
    neg_vector = self.vectorize(neg_batch) # (batch_size x max_len)
    all_passages = torch.cat((pos_vector, neg_vector), dim=0) # (batch_size*2, max_len)

    inner_products = torch.matmul(query_vector, all_passages.T)  # (batch_size x 2*batch_size)

    # normalize probability wrt in-batch negatives (other queries' positives)
    log_probs = F.log_softmax(inner_products, dim=1) # along row/query
    diagonal_scores = torch.diagonal(log_probs)
    log_probs = torch.diag_embed(diagonal_scores)
    ### END YOUR CODE HERE !!!!!

    return log_probs

  def fit(self, train_data, dev_data, model_file, n_epochs=10, batch_size = 16):
    dev_data_iterator = list(make_batch_iterator(dev_data, 4))
    criterion = nn.NLLLoss()
    optimizer = optim.Adam(self.parameters(), lr=5e-5)
    best_recall = 0.
    for epoch in range(n_epochs):
      recall_at_5 = evaluate(dev_data_iterator, self, K=[5], verbose=True)[0]
      if recall_at_5 > best_recall:
        print(
            "Obtained a new best recall@5 of {:.2f}, saving model "
            "checkpoint to {}...".format(recall_at_5, model_file))
        torch.save(self.state_dict(), model_file)
        best_recall = recall_at_5
      self.train()
      running_loss = 0.
      train_data_iterator = list(make_batch_iterator(train_data, batch_size, shuffle=True))
      print("LENGTH OF TRAINING DATA ITERATOR")
      print(len(train_data_iterator))
      print(len(train_data))
      pbar = tqdm.tqdm(enumerate(train_data_iterator))
      for i, (query_batch, passage_batch, example_batch) in pbar:
        optimizer.zero_grad()
        n = passage_batch.shape[0]

        # `query_batch`: (batch_size x max len) A batch of query indices.
        # `passages_batch`: (batch_size x num_psgs x max_len) A batch of passages for each query.
        # `example_batch`: (batch_size) A list of dict,
         # {..., 'positives': [] }  containing indices of positive passages.

        # Sample a random positive and a random negative for each question,
        # calculate nll loss based on where the positive is.
        # Think carefully about what the `target` argument should look like! We
        # have already initialized the NLLLoss() `criterion` for you above.
        ### YOUR CODE HERE !!!!!
        pos_passages_batch = torch.zeros(passage_batch.shape[0], passage_batch.shape[2])
        neg_passages_batch = torch.zeros(passage_batch.shape[0], passage_batch.shape[2])
        targets = torch.zeros(passage_batch.shape[0])
        for i in range(query_batch.shape[0]): # for each query/batch index
          pos_idx = example_batch[i]['positives'] # indices of positive passages for query i
          neg_idx = [idx for idx in range(passage_batch.shape[1]) if idx not in pos_idx]
          rand_pos_idx = pos_idx[torch.randint(len(pos_idx), (1,)).item()]
          if len(neg_idx) == 0:
            rand_neg_idx = pos_idx[torch.randint(len(pos_idx), (1,)).item()]
          else:
            rand_neg_idx = neg_idx[torch.randint(len(neg_idx), (1,)).item()]
          rand_pos_passage = passage_batch[i, rand_pos_idx, :]
          rand_neg_passage = passage_batch[i, rand_neg_idx, :]
          targets[i] = i
          pos_passages_batch[i] = rand_pos_passage
          neg_passages_batch[i] = rand_neg_passage

        targets = targets.to(dtype=torch.long, device=query_batch.device)
        logits = self.forward(query_batch, pos_passages_batch, neg_passages_batch) # (batch_size, batch_size)
        logits = logits.to(device=query_batch.device)
        loss = criterion(logits, targets)

        ### END YOUR CODE HERE !!!!!

        loss.backward()
        optimizer.step()
        lr = optimizer.param_groups[0]['lr']

        # print statistics
        running_loss += loss.item()
        pbar.set_description("Epoch: {}, Loss: {:.2f}, Best R@5: {:.3f}, lr: {:.2g}".format(epoch, running_loss / (i + 1), best_recall, lr))
    print("Reloading best model checkpoint from {}...".format(model_file))
    self.load_state_dict(torch.load(model_file))

You should able to get a recall@5 of 0.76 and recall@20 of 0.91 with hard negatives, but you might need several runs to avoid random worse results.

# Experimentation: 1-Page Report

Now it's time for you to experiment.  Try to improve the recall on the validation set further. Note that we will mainly check the recall@5 on gradescope. Feel free to modify the code above directly or copy it in new cells below.

Hyper-parameter tuning, learning rate scheduling, or adding regularization might not be as helpful (as in previous assignments) for the DPR models, because they were pre-trained and the fine-tuning is only meant to adapt the model slightly.

Here are some substantial and open-ended ideas to try out. You might want to refer to the [DPR paper](https://arxiv.org/abs/2004.04906) as well as [this paper](https://arxiv.org/abs/2005.00181) from Luan et al which provides some intuitions about sparse and dense retrieval.
- **Harder negative examples**, in the hard negative selection we have tried, simply broadly related passages were sampled. We can further experiment with other heuristics to sample **harder** nagatives.
  - The simplest way is perhaps increasing the number of hard negatives. Previously we only include one hard negative per example. Note that this hyper-parameter change will be considered a less interesting approach.
  - We can utilize the model's own predictions, especially those where the model assigns high relevance scores to incorrect passages. These instances, where the model is mistaken but confident, represent valuable hard negatives because they highlight the model's vulnerabilities.
  - We can utilize sparse retrieval model to fetch top-ranking passages for a query.
  - Does a training curriculum (i.e., from simple to hard) help with the optimization?
  - **Note** that you can manually write or collect additional documents from the web for this purpose. You cannot use additional labeled positives.
- **Hybrid retrieval systems** with both sparse and dense retrieval. These systems can capture different aspects of the query-document relations (you might want to explain why). For example, you can explore ensemble approach that takes the scores produced by both retrieval methods to form an aggregated results. The aggregation can be defined by manually selected hyper-parameter or trained parameters.
- **Cross-encoder** to allow the query and context to attend each other. The DPR model above encodes the query and context separately -- this is essential for efficiency when implementing retrievers which need to work on millions of documents. But for just reranking, it is ok to encode the documents and queries together. See [Humeau et al., 2019](https://arxiv.org/abs/1905.01969) (Section 4.2-4.3) for a detailed comparison on the cross-encoder and the bi-encoder architecture implemented above.

For fair comparison, you cannot use any pre-trained language model other than BERT-Mini.

For this section, you will submit a write-up describing the extensions and/or modifications that you tried.  Your write-up should be **1-page maximum** in length and should be submitted in PDF format.  You may use any editor you like, but we recommend using LaTeX and working in an environment like Overleaf.
For full credit, your write-up should include:
1.   A concise and precise description of the extension that you tried.
2.   A motivation for why you believed this approach might improve your model.
3.   A discussion of whether the extension was effective and/or an analysis of the results.  This will generally involve some combination of tables, learning curves, etc.
4.   A bottom-line summary of your results comparing the scores of your improvement to the original model.
The purpose of this exercise is to experiment, so feel free to try/ablate multiple of the suggestions above as well as any others you come up with!
When you submit the file, please name it `report.pdf`.



In [ ]:
class HardNegativeDualEncoderModel(PretrainedDualEncoderModel):
  def __init__(self):
    super().__init__()
    self.hidden_dim = 256
    self.scoring_layer = nn.Linear(self.hidden_dim, 1).to(device)

  def forward(self, query_batch, all_passages_batch, L):
    """Encode queries and passages and compute log probabilities.

    Args:
      query_batch: (batch_size x max_len) token indices for the queries.
      passage_batch: (batch_size x max_len) token indices for the passages.
      L: number of negative passages -- L+1 total passages

    Returns:
      log_probs: (batch_size x batch_size) normalized log probabilities obtained by
        taking a softmax of the inner products between each query and all of
        the passages.
    """
    # Run the forward pass on queries and passages.
    # You will probably want to use the `log_softmax` function to compute the log
    # probabilities once you have the scores.

    batch_size = query_batch.shape[0]
    queries = query_batch.repeat_interleave(L+1, dim=0) # ((L+1)*batch_size, len_q)

    # pos_concat_vector = torch.cat((query_batch, pos_passage_batch), dim=1)  # (L*batch_size, len_p+len_q)
    # neg_concat_vector = torch.cat((query_batch, neg_passage_batch), dim=1)  # (L*batch_size, len_p+len_q)
    # all_concat_vector = torch.cat((pos_concat_vector, neg_concat_vector), dim=0) # (2*L*batch_size, len_q+len_p)
    all_concat_vector = torch.cat((queries, all_passages_batch), dim=1) # ((L+1)*batch_size, len_q+len_p)
    all_embeddings = self.vectorize(all_concat_vector.to(dtype=torch.long)).to(device) # ((L+1)*batch_size, emb_dim)

    # scoring
    scores = self.scoring_layer(all_embeddings) # ((L+1)*batch_size, 1)
    scores = scores.reshape(batch_size, L+1) # (batch_size, L+1)

    # normalize probability wrt in-batch negatives (other queries' positives)
    log_probs = F.log_softmax(scores, dim=1) # along row/query; (batch_size, 2*L)

    return log_probs

  def fit(self, train_data, dev_data, model_file, n_epochs=10, batch_size = 16):
    dev_data_iterator = list(make_batch_iterator(dev_data, 4))
    criterion = nn.NLLLoss()
    optimizer = optim.Adam(self.parameters(), lr=5e-5)
    best_recall = 0.
    for epoch in range(n_epochs):
      recall_at_5 = evaluate(dev_data_iterator, self, K=[5], verbose=True)[0]
      if recall_at_5 > best_recall:
        print(
            "Obtained a new best recall@5 of {:.2f}, saving model "
            "checkpoint to {}...".format(recall_at_5, model_file))
        torch.save(self.state_dict(), model_file)
        best_recall = recall_at_5
      self.train()
      running_loss = 0.
      train_data_iterator = list(make_batch_iterator(train_data, batch_size, shuffle=True))
      pbar = tqdm.tqdm(enumerate(train_data_iterator))
      for i, (query_batch, passage_batch, example_batch) in pbar:
        optimizer.zero_grad()
        n = passage_batch.shape[0] # batch size
        L = 3 # number of negatives and positives each (total 6)

        # `query_batch`: (batch_size x max len) A batch of query indices.
        # `passages_batch`: (batch_size x num_psgs x max_len) A batch of passages for each query.
        # `example_batch`: (batch_size) A list of dict,
         # {..., 'positives': [] }  containing indices of positive passages.

        # Sample a random positive for each query to get batch_size passages,
        # a positive to one query is a negative to the other query
        # calculate nll loss based on where the positive is among the batch.
        # Think carefully about what the `target` argument should look like! We
        # have already initialized the NLLLoss() `criterion` for you above.
        ### YOUR CODE HERE !!!!!
        # pos_passages_batch = torch.zeros(passage_batch.shape[0]*L, passage_batch.shape[2]) # (batch_size*L, max_len_p)
        # neg_passages_batch = torch.zeros(passage_batch.shape[0]*L, passage_batch.shape[2]) # (batch_size*L, max_len_p)
        all_passages_batch = torch.zeros(passage_batch.shape[0]*(L+1), passage_batch.shape[2]) # (batch_size*(L+1), max_len_p)
        targets = torch.zeros(passage_batch.shape[0])
        for i in range(query_batch.shape[0]): # for each query/batch index
          pos_idx = example_batch[i]['positives'] # indices of positive passages for query i
          rand_pos_idx = pos_idx[torch.randint(len(pos_idx), (1,)).item()]
          #pos_passage = passage_batch[i, rand_pos_idx, :] # (1, max_len_p)
          pos_passage = torch.zeros(passage_batch.shape[2])
          neg_idx = [idx for idx in range(passage_batch.shape[1]) if idx not in pos_idx]
          if len(neg_idx) == 0:
            rand_neg_idxs = list(np.random.randint(low=0, high=len(pos_idx)-1, size=L))
            neg_i = [pos_idx[j] for j in rand_neg_idxs]
            neg_passages = passage_batch[i, neg_i, :]
          elif len(neg_idx) < L:
            num_pos = L - len(neg_idx)
            rand_neg_idxs = list(np.random.randint(low=0, high=len(pos_idx)-1, size=num_pos))
            neg_i = [pos_idx[j] for j in rand_neg_idxs]
            neg_passages = torch.cat((passage_batch[i, neg_idx, :], passage_batch[i, neg_i, :]), dim=0)
          else:
            rand_neg_idxs = list(np.random.randint(low=0, high=len(neg_idx)-1, size=L))
            neg_i = [neg_idx[j] for j in rand_neg_idxs]
            neg_passages = passage_batch[i, neg_i, :] # (L, max_len_p)

          all_passages_batch[i*(L+1):(i+1)*(L+1)] = torch.cat((pos_passage.unsqueeze(0), neg_passages), dim=0) # ((L+1)*batch_size, max_len_p)

        # print(all_passages_batch[:,0])
        # print()
        targets = targets.to(dtype=torch.long, device=query_batch.device)
        logits = self.forward(query_batch, all_passages_batch, L).to(device=query_batch.device) # (batch_size, batch_size)
        loss = criterion(logits, targets)
        ### END YOUR CODE HERE !!!!!

        loss.backward()
        optimizer.step()
        lr = optimizer.param_groups[0]['lr']

        # print statistics
        running_loss += loss.item()
        pbar.set_description("Epoch: {}, Loss: {:.2f}, Best R@5: {:.3f}, lr: {:.2g}".format(epoch, running_loss / (i + 1), best_recall, lr))
    print("Reloading best model checkpoint from {}...".format(model_file))
    self.load_state_dict(torch.load(model_file))

In [ ]:
# Train.
hardneg_reranker = HardNegativeDualEncoderModel()
hardneg_reranker.fit(train_data, dev_data, "hardneg_dualencoder.pt")

# Evaluate on the dev set.
_ = evaluate(make_batch_iterator(dev_data, 4), hardneg_reranker, verbose=True)

save_predictions(make_batch_iterator(test_data, 4), hardneg_reranker, "preds_hardneg.txt")

In [ ]:
# trying triple loss function and a variety of difficulty levels for the negative samples
class HardNegativeDualEncoderModel(PretrainedDualEncoderModel):
  def __init__(self):
    super().__init__()

  def forward(self, query_batch, pos_batch, neg_batch):
    """Encode queries and passages and compute log probabilities.

    Args:
      query_batch: (batch_size x max_len) token indices for the queries.
      pos_batch: (batch_size x max_len) token indices for a single positive
        passage per query.
      neg_batch: (batch_size*L x max_len) token indices for a single negative
        passage per query.

    Returns:
      log_probs: (batch_size x batch_size*L) normalized log probabilities obtained by
        taking a softmax of the inner products between the queries and each of
        the positive and negative passages across the batch.
    """
    # Run the forward pass on queries and passages.

    # You will probably want to use the `log_softmax` function to compute the log
    # probabilities once you have the scores.
    query_vector = self.vectorize(query_batch.to(dtype=torch.long)) # (batch_size x max_len)
    pos_vector = self.vectorize(pos_batch.to(dtype=torch.long)) # (batch_size x max_len)
    neg_vector = self.vectorize(neg_batch.to(dtype=torch.long)) # (batch_size*L x max_len)
    all_passages = torch.cat((pos_vector, neg_vector), dim=0) # (batch_size*(L+1), max_len)

    inner_products = torch.matmul(query_vector, all_passages.T)  # (batch_size x batch_size*(L+1))

    # normalize probability wrt in-batch negatives (other queries' positives)
    log_probs = F.log_softmax(inner_products, dim=1) # along row/query
    diagonal_scores = torch.diagonal(log_probs)
    log_probs = torch.diag_embed(diagonal_scores)

    return log_probs

  def fit(self, train_data, dev_data, model_file, n_epochs=10, batch_size = 16):
    dev_data_iterator = list(make_batch_iterator(dev_data, 4))
    criterion = nn.NLLLoss()
    optimizer = optim.Adam(self.parameters(), lr=5e-5)
    best_recall = 0.
    for epoch in range(n_epochs):
      recall_at_5 = evaluate(dev_data_iterator, self, K=[5], verbose=True)[0]
      if recall_at_5 > best_recall:
        print(
            "Obtained a new best recall@5 of {:.2f}, saving model "
            "checkpoint to {}...".format(recall_at_5, model_file))
        torch.save(self.state_dict(), model_file)
        best_recall = recall_at_5
      self.train()
      running_loss = 0.
      train_data_iterator = list(make_batch_iterator(train_data, batch_size, shuffle=True)) # 1125 queries, 71 batches (70*16 + 5) ==> 112500 passages
      pbar = tqdm.tqdm(enumerate(train_data_iterator))

      for i, (query_batch, passage_batch, example_batch) in pbar:
        optimizer.zero_grad()
        n = passage_batch.shape[0]
        L = 2 # number of negatives
        if i <= 13: # first - 14
          nh = 0
        elif i <= 27: # second - 28
          nh = 1
        elif i <= 41: # third - 42
          nh = 2
        else: # last 29
          nh = 2
        ns = L - nh

        ### YOUR CODE HERE !!!!!
        pos_passages_batch = torch.zeros(passage_batch.shape[0], passage_batch.shape[2]) # (batch_size x max_len)
        L_neg_passages_batch = torch.zeros(passage_batch.shape[0]*(nh + ns), passage_batch.shape[2]) # (batch_size*L x max_len)
        targets = torch.zeros(passage_batch.shape[0]) # (batch_size, )
        query_embeddings = self.vectorize(query_batch.to(dtype=torch.long)) # (batch_size x max_len)
        triplet_loss_mean_sum=0
        for i in range(query_batch.shape[0]): # for each query/batch index
          # add to targets
          targets[i] = i

          # select 1 random positive passage
          pos_idx = example_batch[i]['positives'] # indices of all positive passages for query i
          positive_passages = passage_batch[i,pos_idx,:]
          rand_pos_idx = pos_idx[torch.randint(len(pos_idx), (1,)).item()]
          rand_pos_passage = passage_batch[i, rand_pos_idx, :]
          pos_passages_batch[i] = rand_pos_passage # selected positive passage; not an embedding

          # get relevance scores for all negative passages
          neg_idx = [idx for idx in range(passage_batch.shape[1]) if idx not in pos_idx] # indices of all neg passages for query i
          if len(neg_idx) == 0: # no negative passages at all - use positive passages for everything
            hard_neg_passages = positive_passages[:nh]
            if ns != 0:
              easy_neg_passages = positive_passages[nh:nh+ns]
              total = torch.cat((hard_neg_passages, easy_neg_passages), dim=0)
            else:
              total = hard_neg_passages

          # sort negatives based on high relevance scores and gather sorted negative passages batch
          curr_neg_embeddings = self.vectorize(passage_batch[i,neg_idx,:].to(dtype=torch.long))
          scores = torch.matmul(query_embeddings[i], curr_neg_embeddings.T) # (1, len(negatives)) similarity scores
          hard_neg_indices = torch.argsort(scores, descending=True).to('cpu')  # passage indices of sorted negative scores
          neg_passages_batch = passage_batch[i, hard_neg_indices,:] # all negative passages for query i, not embeddings

          # get batches of hard and easy negative passages based on current ns and nh
          if len(neg_passages_batch) >= L: # enough neg passages for hard negs and easy negs
            hard_neg_passages = neg_passages_batch[:nh,:] # none of these are embeddings
            if ns != 0:
              easy_neg_passages = neg_passages_batch[-1*ns:,:]
              total = torch.cat((hard_neg_passages, easy_neg_passages), dim=0)
            else:
              total = hard_neg_passages
          elif len(neg_passages_batch) > nh: # more than enough neg passages for hard negs, but not enough for easy negs
            hard_neg_passages = neg_passages_batch[:nh,:]
            remaining_negs = ns - len(neg_passages_batch[nh:,:]) # number of positive passages to fill in for easy negs
            if ns != 0:
              easy_neg_passages = torch.cat((neg_passages_batch[nh+1:,:],positive_passages[:remaining_negs,:]),dim=0)
              total = torch.cat((hard_neg_passages, easy_neg_passages), dim=0)
            else:
              total = hard_neg_passages
          elif len(neg_passages_batch) == nh: # exactly enough for hard negs, 0 for easy negs
            hard_neg_passages = neg_passages_batch
            if ns != 0:
              easy_neg_passages = positive_passages[:ns,:]
              total = torch.cat((hard_neg_passages, easy_neg_passages), dim=0)
            else:
              total = hard_neg_passages

          elif len(neg_passages_batch) <= nh and len(neg_passages_batch)>0: # not enough for hard negs, 0 for easy negs
            remaining_hard_negs = nh - len(neg_passages_batch)
            hard_neg_passages = torch.cat((neg_passages_batch,positive_passages[:remaining_hard_negs,:]),dim=0)
            if ns != 0:
              easy_neg_passages = positive_passages[remaining_hard_negs:remaining_hard_negs+ns,:]
              total = torch.cat((hard_neg_passages, easy_neg_passages), dim=0)
            else:
              total = hard_neg_passages

          L_neg_passages_batch[i*L:(i+1)*L] = total # (L x max_len)

        positive_embeddings = self.vectorize(pos_passages_batch.to(dtype=torch.long)) # (batch_size x max_len)
        negative_embeddings = self.vectorize(L_neg_passages_batch.to(dtype=torch.long)) # (batch_size*L x max_len)

        targets = targets.to(dtype=torch.long, device=query_batch.device) # (batch_size, )
        logits = self.forward(query_batch, pos_passages_batch, L_neg_passages_batch).to(device=query_batch.device) # (batch_size, batch_size*(L+1))

        triplet_loss = F.triplet_margin_loss(query_embeddings.repeat_interleave(L).reshape(query_batch.shape[0]*L, -1),
                                             positive_embeddings.repeat_interleave(L).reshape(query_batch.shape[0]*L, -1),
                                             negative_embeddings,
                                             margin = 1).to(device)
        loss = criterion(logits, targets).to(device)
        loss += triplet_loss

        ### END YOUR CODE HERE !!!!!

        loss.backward()
        optimizer.step()
        lr = optimizer.param_groups[0]['lr']

        # print statistics
        running_loss += loss.item()
        pbar.set_description("Epoch: {}, Loss: {:.2f}, Best R@5: {:.3f}, lr: {:.2g}".format(epoch, running_loss / (i + 1), best_recall, lr))
    print("Reloading best model checkpoint from {}...".format(model_file))
    self.load_state_dict(torch.load(model_file))

In [ ]:
# Train.
hardneg_reranker = HardNegativeDualEncoderModel()
hardneg_reranker.fit(train_data, dev_data, "hardneg_dualencoder.pt")

# Evaluate on the dev set.
_ = evaluate(make_batch_iterator(dev_data, 4), hardneg_reranker, verbose=True)

save_predictions(make_batch_iterator(test_data, 4), hardneg_reranker, "preds_hardneg.txt")

  0%|          | 0/29 [00:00<?, ?it/s]

K	5
------------
Rec@K	0.595
Obtained a new best recall@5 of 0.59, saving model checkpoint to hardneg_dualencoder.pt...


0it [00:00, ?it/s]

  0%|          | 0/29 [00:00<?, ?it/s]

K	5
------------
Rec@K	0.707
Obtained a new best recall@5 of 0.71, saving model checkpoint to hardneg_dualencoder.pt...


...

0it [00:00, ?it/s]

Reloading best model checkpoint from hardneg_dualencoder.pt...


0it [00:00, ?it/s]

K	5	20
------------------------
Rec@K	0.802	0.914
Saving predictions to preds_hardneg.txt


0it [00:00, ?it/s]

In [ ]:
# Evaluate on the dev set.
_ = evaluate(make_batch_iterator(dev_data, 4), hardneg_reranker, verbose=True)

save_predictions(make_batch_iterator(test_data, 4), hardneg_reranker, "preds_hardneg.txt")

0it [00:00, ?it/s]

K	5	20
------------------------
Rec@K	0.802	0.914
Saving predictions to preds_hardneg.txt


0it [00:00, ?it/s]